### Code that obtains stellar, gas and residual velocity maps alongside resiudal velocity value for a set of galaxies

Import all necessary libraries, including extra code needed to run smoothly

In [ ]:
import numpy as np
import marvin
from marvin import config
import pandas as pd
from marvin.tools import Maps
from astropy.io import fits
import matplotlib.pyplot as plt

# Set Marvin release to DR17 and avoid API error
config.setDR('DR17')
config.switchSasUrl(sasmode='mirror')

Define function to calculate the masked resiudal velocity of a galaxy

In [ ]:
def calculate_delta_v_star_g(stellar_vel, gas_vel):
    """
    
    Calculates the absolute difference in the stellar and gas velocity at each point in a maps as follows:
    
    |Δ_v-g| = |stellar_vel(x, y) - gas_vel(x, y)|
    
    Also calculates the sum of the residual map at each point to give a value to quantify the offset 
    between the gas and stellar velocity
    
    INPUT
    stellar_vel: stellar velocity map values
    gas_vel: gas velocity map values
    
    OUTPUT
    dvsg_map: absolute map of difference between stellar and gas velocity
    dvsg_sum: sum of dvsg at each point on the dvsg map
    """
    
    # Create residual map
    dvsg_map = np.abs(stellar_vel - gas_vel)
    
    # Calculate summed residual value
    dvsg_sum = np.sum(dvsg_map)
    
    return dvsg_map, dvsg_sum

Access .txt file containing set of 100 selected MaNGA candidates that satisfy selection criteria

In [ ]:
candidates = np.loadtxt("100_galaxy_plateifu_list.txt", dtype='int')  # Access as integer

Open DRPall fits file

In [ ]:
# Open data
drpall = fits.open('/Users/Jonah/sas/dr17/manga/spectro/redux/v3_1_1/drpall-v3_1_1.fits')
tbdata = drpall[1].data

Loop over all selected galaxies to obtain stellar, gas and residual maps and residual value

In [ ]:
# Create lists to store values
stellar_vel_map_list = []
gas_vel_map_list = []
dvsg_map_list = []
dvsg_sum_list = []

# Run over all galaxies in sample and add each value to respective list
for i in range(len(candidates)):
    
    # Access plateifu of random id
    plateifu = tbdata["plateifu"][candidates[i]]
    
    print(i)
    print("PlateIFU: ", plateifu)
    
    # Use Marvin to load in ith map
    maps = Maps(str(plateifu), download=False)  # Data already downloaded to local drive
    
    # Obtain MASKED stellar and gas velocities
    sv_map = maps.stellar_vel.masked
    gv_map = maps.emline_gvel_ha_6554.masked
    
    # Pass though residual function to obtain residual map and summed residual value
    dvsg_out = calculate_delta_v_star_g(stellar_vel=sv_map, gas_vel=gv_map)
    dvsg_map = dvsg_out[0]
    dvsg_sum = dvsg_out[1]
    
    # Append data to lists
    stellar_vel_map_list.append(sv_map)
    gas_vel_map_list.append(gv_map)
    dvsg_map_list.append(dvsg_map)
    dvsg_sum_list.append(dvsg_sum)

Create a DataFrame directly from the lists using a dictionary

In [ ]:
df = pd.DataFrame({
    'sv_maps': stellar_vel_map_list,
    'gv_maps': gas_vel_map_list,
    'dvsg_maps': dvsg_map_list,
    'dvsg_sums': dvsg_sum_list
})

df.head()

In [ ]:
print(dvsg_sum_list)

### Analyse sample

Plot first 3 galaxies in sample

In [ ]:
# Create figure
fig, ax = plt.subplots(nrows=3, ncols=3, figsize=(15, 15))

for i in range(3):

    # Access ith row of data
    sv = df['sv_maps'][i].data
    gv = df['gv_maps'][i].data
    dvsg = df['dvsg_maps'][i].data
    dvsg_sum = df['dvsg_sums'][i]

    # Print plateifu to verify with Marvin Web
    print("Plateifu: ", tbdata["plateifu"][candidates[i]])

    # Generate x and y coordinates for plotting
    x = np.arange(np.shape(sv)[0])
    y = x

    # Plot sv colormap
    sv_pcolormesh = ax[i, 0].pcolormesh(x, y, sv, shading="auto", cmap="RdBu_r")
    fig.colorbar(sv_pcolormesh, ax=ax[i, 0])
    ax[i, 0].set_aspect('equal')
    ax[i, 0].set_title("Stellar velocity")

    # Plot gv colormap
    gv_pcolormesh = ax[i, 1].pcolormesh(x, y, gv, shading="auto", cmap="RdBu_r")
    fig.colorbar(gv_pcolormesh, ax=ax[i, 1])
    ax[i, 1].set_aspect('equal')
    ax[i, 1].set_title("Gas velocity")

    # Plot dvsg colormap
    dvsg_pcolormesh = ax[i, 2].pcolormesh(x, y, dvsg, shading="auto", cmap="RdBu_r")
    fig.colorbar(dvsg_pcolormesh, ax=ax[i, 2])
    ax[i, 2].set_aspect('equal')
    ax[i, 2].set_title(r"$|\Delta V_{\star-g}|$ = " + "{:.2f}".format(dvsg_sum));